In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
!pip install -q openai #--progress-bar off

In [3]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [4]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [5]:
output_datapath = "data/output/2023.11.21-3_Llama2_7B_API_7-few-shot/"

In [6]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [7]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [8]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) |
        (data_sel_4_few_shot_examples.text_id==370)  |
        (data_sel_4_few_shot_examples.text_id==1422) |
        (data_sel_4_few_shot_examples.text_id==4329) |
        (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
250,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000
986,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500
3026,4329,10.18356/30e5b8d3-en,With the introduction of FSC standards in the ...,15,0.0,9.0,1.000000
3444,4929,10.18356/5e8977eb-en,"Hence, climate-economy models without explicit...",13,0.0,3.0,1.000000
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000


In [9]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [10]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [11]:
template_part1 = """[INST]
<<SYS>>
In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 7 examples of expected classification given an input text:

"""+few_shots+"""

The text to be processed is:
<</SYS>>

'''
"""

template_part2 = """
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [12]:
data_sel = data #.sample(2)

In [15]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [16]:
import time

cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    query = template_part1 + text + template_part2
    
    body = {
      "model": "meta-llama/Llama-2-7b-chat-hf",
      "messages": [
          {"role": "system", "content": "You are a helpful assistant."}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
    result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
    
    try:
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    except Exception as e:
        print("#############################################################################")
        print(e)
        print("#############################################################################")
        
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_7B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

0 6668 0   Based on the provided text, I would classify it under SDG 8 - Decent Work and Economic Growth.

The text mentions the resumption of service on the L train after it derailed from the track in Canarsie, Brooklyn, and the adequacy of capital flows to cover the current account deficit. These details suggest that the text is related to economic growth and development, which is a key aspect of SDG 8.
1 6099 0   ---SDG 8---
2 6471 0   ---SDG 16---
3 1786 0   Sure, I can help you with that! Based on the given text, here are the possible labels for each section:

---SDG 8---

The text mentions "capital flows have been adequate to cover the current account deficit thus far." This suggests that the text is related to economic growth and financial flows, which is aligned with SDG 8 (Decent Work and Economic Growth).

---SDG 12---

The text mentions "productive forestlands that are suitable for afforestation amount to about 166,192.5 ha." This suggests that the text is related to sustain

34 6228 0   ---SDG 8---
35 3148 0   Based on the provided text, I would classify it under SDG 8 - Decent Work and Economic Growth. The text mentions the trade and current account deficits, which suggests that there are issues with the country's economic growth and supply bottlenecks. This aligns with the goals of SDG 8, which aims to promote sustained, inclusive and sustainable economic growth, full and productive employment, and decent work for all.
36 2298 0   Based on the given text, I would classify it under SDG 3 - Good Health and Well-being. The text mentions a hospital's pediatric intensive care unit and deficiencies found in the unit, which suggests that the facility is not providing adequate healthcare services to children. This aligns with SDG 3's goal of ensuring healthy lives and promoting well-being for all at all ages. Therefore, the classification is ---SDG 3---.
#############################################################################
'choices'
#####################

81 2391 1   ---SDG 8---
82 841 1   ---SDG 1---
83 1373 1   Based on the given text, I would classify it under SDG 1. No poverty.

The text discusses the concept of affordability and its impact on the perception of what is considered necessary or not. It suggests that as incomes increase, what is considered normal may also increase, and vice versa. This aligns with the SDG 1 target of reducing poverty and promoting equitable economic growth. Therefore, the text can be classified under SDG 1.
84 5239 1   ---SDG 1---
85 1086 1   ---SDG 8---
86 330 1   Sure, I can help you with that! Based on the given text, I would classify it under SDG 16 - Peace, Justice and Strong Institutions. Here's why:

The text discusses the correlation between government spending on protecting families and overall child poverty rates, indicating that some countries are making more progress than others towards creating a just society. This aligns with the SDG 16's focus on promoting peaceful and inclusive societie

133 98 2   ---SDG 2---
134 3973 2   ---SDG 2---
135 4844 2   ---SDG 2---
136 2970 2   ---SDG 12---
137 2747 2   ---SDG 15---
138 3976 2   ---SDG 8---
139 2988 2   ---SDG 12---
140 2775 2   ---SDG 2---
141 5423 2   ---SDG 8---
142 5748 2   ---SDG 15---
143 1642 2   Sure, I can help you with that! Based on the given text, I would classify it under SDG 2 - Zero Hunger. Here's why:

The text mentions "ordinary food businesses" and "food bank," which suggests that the text is related to food security and access to food. The text also mentions "retail food business operators" supplying food to a food bank, which further reinforces the connection to SDG 2.

Therefore, I would label the given text as ---SDG 2---.
144 2727 2   ---SDG 8---
145 869 2   ---SDG 2---
146 3205 2   ---SDG 2---
147 6506 2   ---SDG 8---
148 1622 2   ---SDG 2---
149 2827 2   Based on the provided text, I would classify it under SDG 8 - Decent Work and Economic Growth.
150 223 2   Great! Based on the given text, I would c

193 1028 3   ---SDG 3---
194 6378 3   ---SDG 3---
195 1869 3   Based on the given text, I would classify it under SDG 3 - Good Health and Well-being. The text mentions "STI incidence rates" and "morbidity of trichomoniasis", which are both related to sexual and reproductive health, a key area of focus under SDG 3.
196 1372 3   Based on the given text, I would classify it under SDG 3 - Good Health and Well-being. The text discusses various factors that affect healthcare expenditures, including smoking, obesity, physical health status, and mortality. These factors are all related to the health and well-being of individuals, which is the primary focus of SDG 3. Therefore, the text can be classified as ---SDG 3---.
197 3450 3   ---SDG 3---
198 1807 3   ---SDG 3---
199 1124 3   ---SDG 3---
200 1131 3   ---SDG 3---
201 4254 3   Sure, I can help you with that! Based on the given text, I would classify it under SDG 8 - Decent Work and Economic Growth. Here's why:

The text mentions the trade a

...

718 4937 11   ---SDG 11---
719 6039 11   ---SDG 12---
720 5667 12   ---SDG 12---
721 5041 12   ---SDG 8---
722 3004 12   ---SDG 12---
723 2110 12   Great! Based on the given text, I would classify it under SDG 12 - Responsible Consumption and Production. The text highlights the initiative of chemical companies in Colombia to improve their environmental, health, and safety performance, and to foster responsible management of chemicals. This aligns with the SDG 12 target to promote sustainable consumption and production patterns, and to reduce waste and the ecological footprint of consumption and production.
724 6 12   Sure, I can help you with that! Based on the given text, I would classify it under SDG 12 - Responsible Consumption and Production. The text discusses the issue of waste management in different economies, highlighting the importance of recycling and reducing waste in countries with rising incomes. It also mentions the need for a market for recycled goods, which aligns with

793 2575 13   Great! Based on the provided text, I would classify it under SDG 13: Climate Action.
794 5881 13   ---SDG 11---
795 2540 13   ---SDG 16---
796 6671 13   ---SDG 17---
797 4343 13   ---SDG 13---
798 1964 13   ---SDG 15---
799 1620 13   ---SDG 13---
800 3379 13   ---SDG 17---
801 4843 13   ---SDG 16---
802 6675 13   ---SDG 17---
803 6696 13   ---SDG 13---
804 5030 13   ---SDG 17---
805 5047 13   ---SDG 8---
806 220 13   ---SDG 12---
807 1613 13   ---SDG 12---
808 1863 13   Great! Based on the provided text, I would classify it under SDG 8 - Decent Work and Economic Growth.

---SDG 8---
809 5441 13   ---SDG 12---
810 3560 13   ---SDG 9---
811 5575 13   Sure, I can help you with that! Based on the given text, I would classify it under SDG 16 - Peace, Justice and Strong Institutions. Here's why:

The text mentions the importance of networking and collaboration between different groups to avoid duplication and foster complementarity and synergy between activities. This aligns wi

871 71 14   ---SDG 17---
872 5815 14   Sure, I can help you with that! Based on the given text, I would classify it under SDG 13 - Climate Action.
873 1388 14   Sure, I can help you with that! Based on the given text, I would classify it under SDG 8 - Decent Work and Economic Growth. Here's why:

The text discusses the characterization of small-scale fisheries (SSF) and the socio-economic survey being carried out in the Mediterranean and the Black Sea. It mentions that large-scale fisheries are associated with high capital costs and sophisticated technology, while SSF are associated with small vessels and labor-intensive fishing techniques. However, the text also notes that modern fishing operations have made even certain SSF more advanced technologically and have a high level of fishing effort.

This discussion highlights the importance of sustainable and inclusive economic growth in the fishing industry, which is a key aspect of SDG 8. By promoting sustainable fishing practices and s

943 1285 15   ---SDG 17---
944 4219 15   ---SDG 15---
945 3113 15   ---SDG 15---
946 4121 15   ---SDG 15---
947 3264 15   ---SDG 11---
948 2523 15   ---SDG 15---
949 3474 15   ---SDG 6---
950 518 15   Based on the given text, I would classify it under SDG 15 - Life On Land.

The text mentions the following:

* High degree of uncertainty concerning the status of several species (83 Data Deficient species and 38% of CR species, 36% of EN species, 23% of VU species, 30% of NT species and 26% of LC species)
* 930 endemic species (out of 7,000 floral species)
* Morocco having the highest percentage of endemic reptile species and the highest number of European relict reptile species
* Centers of diversity of certain reptile genera

All of these points relate to the conservation and management of land and its biodiversity, which is the primary focus of SDG 15.
951 2066 15   ---SDG 14---
952 389 15   Based on the given text, I would classify it under SDG 15 - Life On Land.

The text mentions "

989 3167 16   Great! Based on the provided text, I would classify it under SDG 16 - Peace, Justice and Strong Institutions.
990 5026 16   ---SDG 8---
991 2834 16   ---SDG 16---
992 2469 16   ---SDG 16---
993 1609 16   ---SDG 8---
994 5916 16   ---SDG 16---
995 567 16   ---SDG 16---
996 4214 16   ---SDG 16---
997 322 16   ---SDG 8---
998 6351 16   ---SDG 8---
999 4375 16   ---SDG 16---
1000 2522 16   ---SDG 5---
1001 2889 16   ---SDG 8---
1002 3927 16   ---SDG 16---
1003 1248 16   ---SDG 8---
1004 57 16   ---SDG 16---
1005 339 16   ---SDG 8---
1006 4967 16   Great! Based on the provided text, I would classify it under SDG 8 - Decent Work and Economic Growth. The text primarily focuses on the entrepreneurial journey of Strive Masiyiwa in obtaining a mobile telecommunications license in Zimbabwe, highlighting the obstacles he faced from the government and the legal challenges he pursued to obtain the license. The text also touches upon the themes of ethical business practices, the role of

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [20]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(167, 211, 2, '  ---SDG 12---')
(933, 3170, 15, '  ---SDG 8---')
(653, 3420, 10, '  ---SDG 10---')
(519, 4081, 8, '  Great! Based on the given text, I would classify it under SDG 8 - Decent Work and Economic Growth.')
(90, 5668, 1, '  Sure, I can help you with that! Based on the given text, I would classify it under SDG 8 - Decent Work and Economic Growth. The text discusses issues related to economic growth, income inequality, and the impact of policy interventions on growth outcomes, which are all relevant to SDG 8.')
(533, 6656, 8, '  ---SDG 8---')
(483, 3317, 8, '  ---SDG 8---')
(753, 3777, 12, '  ---SDG 12---')
(52, 6243, 0, "  Great! I'm ready to help you classify the given text according to the Sustainable Development Goals (SDGs).\n\n---SDG 8---\n\nThe text primarily deals with economic and political issues, including the implementation of the Uniform Civil Code (UCC), the state of law and order in Delhi and West Bengal, and the involvement of BJP ministers in corruption. There

In [21]:
len(tuple_list)

1019

In [22]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [23]:
results_processed

,cnt,id,true_label,predicted_label
0,167,211,2,---SDG 12---
1,933,3170,15,---SDG 8---
2,653,3420,10,---SDG 10---
3,519,4081,8,"Great! Based on the given text, I would clas..."
4,90,5668,1,"Sure, I can help you with that! Based on the..."
...,...,...,...,...
1014,771,1616,12,---SDG 12---
1015,314,1094,5,---SDG 4---
1016,133,98,2,---SDG 2---
1017,441,1273,7,---SDG 8---


In [24]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Based on the provided text, I would classify..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I can help you with that! Based on the..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 8---
...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 17---
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---


In [25]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Based on the provided text, I would classify...",8
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---,8
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I can help you with that! Based on the...",8
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 8---,8
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---,16
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 17---,17
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---,16
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---,16


In [26]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
1,51,51,51,51,51,51,51,51,51,51,51
10,20,20,20,20,20,20,20,20,20,20,20
11,44,42,44,44,42,42,42,44,44,44,44
12,102,102,102,102,102,102,102,102,102,102,102
13,27,27,27,27,27,27,27,27,27,27,27
14,33,33,33,33,33,33,33,33,33,33,33
15,51,50,51,51,50,50,50,51,51,51,51
16,86,85,86,86,85,85,85,86,86,86,86
17,28,28,28,28,28,28,28,28,28,28,28


In [27]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [28]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.69      0.58      0.63        60
           2       0.86      0.40      0.55        60
           3       0.77      0.88      0.82        60
           4       0.85      0.67      0.75        60
           5       0.94      0.53      0.68        60
           6       0.90      0.62      0.73        60
           7       1.00      0.40      0.57        60
           8       0.17      0.88      0.28        60
           9       0.64      0.23      0.34        60
          10       0.60      0.20      0.30        60
          11       0.64      0.47      0.54        60
          12       0.47      0.80      0.59        60
          13       0.93      0.42      0.57        60
          14       0.88      0.48      0.62        60
          15       0.82      0.70      0.76        60
          16       0.42      0.60      0.49        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.